# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAINheader as train



Chosen class grouping: two-classes


Directory /Users/diego/Desktop/iteso/TOG/ exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/src exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/media exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/scores exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN does not exist. Creating it and continuing withexecution
Directory /Users/diego/Desktop/iteso/TOG/data/PH1/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH2/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing wit

# Model training

## KNN

### Active Hand prediction

In [3]:
train.arch.find_best(sup.DATA_AH_PF, sup.active_hand_col, "two-classes", sup.TRAIN_KNN_CODE)

updating best... 0.8892215568862275
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf does not exist. Creating it and continuing withexecution
updating best... 0.9021956087824351
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.907185628742515
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.908183632734531
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9151696606786427
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9171656686626747
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9191616766467066
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/

### Sign prediction

In [4]:
train.arch.find_best(sup.DATA_S_PF, sup.class_numeric_column, "two-classes", sup.TRAIN_KNN_CODE)

updating best... 0.54296875
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf does not exist. Creating it and continuing withexecution
updating best... 0.55859375
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.560546875
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.572265625
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.57421875
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.583984375
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.587890625
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best.

In [5]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, "two-classes", sup.TRAIN_KNN_CODE)

updating best... 0.6511627906976745
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv does not exist. Creating it and continuing withexecution
updating best... 0.6744186046511628
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.7441860465116279
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.7906976744186046
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.813953488372093
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.8372093023255814
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.8604651162790697
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/S

## BERT

In [6]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, "two-classes", sup.TRAIN_BERT_CODE)

updating best... 0.5116279069767442
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/BERT/Spv/prajjwal1/bert-tiny does not exist. Creating it and continuing withexecution
updating best... 0.6511627906976745
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/BERT/Spv/prajjwal1/bert-tiny exists. Continuing with execution
updating best... 0.6744186046511628
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/BERT/Spv/prajjwal1/bert-tiny exists. Continuing with execution
updating best... 0.6976744186046512
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/BERT/Spv/prajjwal1/bert-tiny exists. Continuing with execution


ValueError: cannot reshape array of size 222372 into shape (12,31)

# SCORE ANALYSIS

In [ ]:
import pandas as pd
from datetime import datetime

## KNN

In [ ]:
knn_scores_df = pd.DataFrame(sup.knn_score_tracker, columns=sup.knn_scores_columns)

In [ ]:
knn_scores_df

In [ ]:
knn_scores_df.sort_values(by="accuracy", ascending=False).head(50)

In [ ]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, "two-classes", sup.TRAIN_KNN_CODE))
knn_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT,"two-classes", sup.TRAIN_KNN_CODE, f"{now}.csv"), index=False)

## BERT

In [ ]:
bert_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)

In [ ]:
bert_scores_df

In [ ]:
bert_scores_df.sort_values(by="accuracy", ascending=False).head(50)

In [ ]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, "two-classes", sup.TRAIN_BERT_CODE))
bert_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, "two-classes", sup.TRAIN_BERT_CODE, f"{now}.csv"), index=False)